# Dementia Analysis

The following Jupyter notebook in Python covers our analysis of dementia data. All related files for this analysis can be found in the `dementia_project` folder.

## Table of Contents

1. [Setup](#setup)
2. [Explore the dataset](#explore)
3. [Clean the dataset](#clean)

## <font color=green> 1. Setup <a name="setup"></a></font> ##

### Import libraries

In [2]:
import pandas as pd
import numpy as np;

/var/folders/_7/qfzsb2m168gd8fj4j2bnvgn00000gn/T/ipykernel_1380/2805347610.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Load the dataset

In [3]:
dementia_data = pd.read_csv("dementia_studies_data.csv", delimiter=",")
dementia_data.head()

,ID,age,gender,dementia,dementia_all,educationyears,EF,PS,Global,diabetes,...,hypercholesterolemia,lacunes_num,fazekas_cat,study,study1,SVD Simple Score,SVD Amended Score,Fazekas,lac_count,CMB_count
0,1,52.67,male,0.0,0,11.0,-2.403333,-1.290000,-1.287000,0,...,Yes,more-than-zero,2 to 3,scans,scans,3.0,7.0,3,>5,>=1
1,10,64.58,male,0.0,0,10.0,1.280000,0.360000,0.744000,0,...,Yes,more-than-zero,0 to 1,scans,scans,2.0,3.0,1,1 to 2,>=1
2,100,74.92,male,0.0,0,8.0,-1.440000,-1.520000,-0.922000,0,...,Yes,more-than-zero,0 to 1,scans,scans,1.0,2.0,1,1 to 2,0
3,101,74.83,male,1.0,1,9.0,NaN,-2.136271,-1.301102,0,...,Yes,more-than-zero,2 to 3,scans,scans,2.0,4.0,2,3 to 5,0
4,102,79.25,male,0.0,0,10.0,-0.920000,-1.493333,-0.924000,0,...,Yes,more-than-zero,2 to 3,scans,scans,2.0,3.0,2,1 to 2,0


## <font color=green> 2. Explore the dataset <a name="explore"></a></font> ##

<div class=\"alert alert-block alert-info\">
    <b>Shape:</b> First, let's find out the shape of the data.
</div>

In [22]:
dementia_data.shape

(1842, 22)

There are 1842 rows in the dataset, which is the number of entities and 22 coloumns, the features. The features consist of 21 independent viables and the feature <font color=green> **dementia** </font> or <font color =green> **dementia_all** </font> will be the dependent viable. 

<font color=green> What is the difference between dementia and dementia_all </font>


The difference between <font color=green> dementia </font> and <font color =green> dementia_all </font> is ??? 


In [4]:
dementia_data

,ID,age,gender,dementia,dementia_all,educationyears,EF,PS,Global,diabetes,...,hypercholesterolemia,lacunes_num,fazekas_cat,study,study1,SVD Simple Score,SVD Amended Score,Fazekas,lac_count,CMB_count
0,1,52.67,male,0.0,0,11.0,-2.403333,-1.290000,-1.287000,0,...,Yes,more-than-zero,2 to 3,scans,scans,3.0,7.0,3,>5,>=1
1,10,64.58,male,0.0,0,10.0,1.280000,0.360000,0.744000,0,...,Yes,more-than-zero,0 to 1,scans,scans,2.0,3.0,1,1 to 2,>=1
2,100,74.92,male,0.0,0,8.0,-1.440000,-1.520000,-0.922000,0,...,Yes,more-than-zero,0 to 1,scans,scans,1.0,2.0,1,1 to 2,0
3,101,74.83,male,1.0,1,9.0,NaN,-2.136271,-1.301102,0,...,Yes,more-than-zero,2 to 3,scans,scans,2.0,4.0,2,3 to 5,0
4,102,79.25,male,0.0,0,10.0,-0.920000,-1.493333,-0.924000,0,...,Yes,more-than-zero,2 to 3,scans,scans,2.0,3.0,2,1 to 2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,989,67.00,female,0.0,0,13.0,-0.100000,-0.020000,-0.260000,0,...,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,0,Zero,0
1838,990,71.00,female,0.0,0,13.0,-0.370000,-1.060000,-1.003333,0,...,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,1,Zero,0
1839,991,55.00,male,0.0,0,10.0,0.460000,0.960000,0.610000,0,...,Yes,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,0,Zero,0
1840,995,54.00,male,0.0,0,10.0,0.190000,0.030000,0.590000,0,...,Yes,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,1,Zero,0


In [5]:
dementia_data.dtypes

ID                        int64
age                     float64
gender                   object
dementia                float64
dementia_all              int64
educationyears          float64
EF                      float64
PS                      float64
Global                  float64
diabetes                  int64
smoking                  object
hypertension             object
hypercholesterolemia     object
lacunes_num              object
fazekas_cat              object
study                    object
study1                   object
SVD Simple Score        float64
SVD Amended Score       float64
Fazekas                   int64
lac_count                object
CMB_count                object
dtype: object

In [6]:
dementia_data.columns


Index(['ID', 'age', 'gender', 'dementia', 'dementia_all', 'educationyears',
       'EF', 'PS', 'Global', 'diabetes', 'smoking', 'hypertension',
       'hypercholesterolemia', 'lacunes_num', 'fazekas_cat', 'study', 'study1',
       'SVD Simple Score', 'SVD Amended Score', 'Fazekas', 'lac_count',
       'CMB_count'],
      dtype='object')

In [7]:
dementia_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1842 entries, 0 to 1841
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1842 non-null   int64  
 1   age                   1842 non-null   float64
 2   gender                1842 non-null   object 
 3   dementia              1808 non-null   float64
 4   dementia_all          1842 non-null   int64  
 5   educationyears        1842 non-null   float64
 6   EF                    1634 non-null   float64
 7   PS                    1574 non-null   float64
 8   Global                1534 non-null   float64
 9   diabetes              1842 non-null   int64  
 10  smoking               1831 non-null   object 
 11  hypertension          1842 non-null   object 
 12  hypercholesterolemia  1842 non-null   object 
 13  lacunes_num           1842 non-null   object 
 14  fazekas_cat           1842 non-null   object 
 15  study                

## 3. Exploring the dataset <a name="clean"></a>

### Missing values

To check whether the dataset is containing missing values, we run the following code: 

Importing various libraries

In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import asarray

In [9]:
dementia_data.isnull().values.any()

True

As the we get a 'True', we now know that there is missing data in the dataset, to check which columns are missing data and how many rows in the columns are having values, we use the count()-method 

In [10]:
dementia_data.count()

ID                      1842
age                     1842
gender                  1842
dementia                1808
dementia_all            1842
educationyears          1842
EF                      1634
PS                      1574
Global                  1534
diabetes                1842
smoking                 1831
hypertension            1842
hypercholesterolemia    1842
lacunes_num             1842
fazekas_cat             1842
study                   1842
study1                  1842
SVD Simple Score        1165
SVD Amended Score       1165
Fazekas                 1842
lac_count               1842
CMB_count               1842
dtype: int64

In [11]:
dementia_data.isna().sum()

ID                        0
age                       0
gender                    0
dementia                 34
dementia_all              0
educationyears            0
EF                      208
PS                      268
Global                  308
diabetes                  0
smoking                  11
hypertension              0
hypercholesterolemia      0
lacunes_num               0
fazekas_cat               0
study                     0
study1                    0
SVD Simple Score        677
SVD Amended Score       677
Fazekas                   0
lac_count                 0
CMB_count                 0
dtype: int64

In [12]:
pd.set_option('display.max_columns', None)
dementia_data[dementia_data["smoking"].isnull()]


,ID,age,gender,dementia,dementia_all,educationyears,EF,PS,Global,diabetes,smoking,hypertension,hypercholesterolemia,lacunes_num,fazekas_cat,study,study1,SVD Simple Score,SVD Amended Score,Fazekas,lac_count,CMB_count
679,1097,78.0,female,0.0,0,10.0,0.28,-1.41,-0.700000,0,NaN,Yes,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,1,Zero,0
736,12,64.0,female,0.0,0,10.0,0.09,-0.31,-0.220000,0,NaN,Yes,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,1,Zero,0
878,1454,73.0,male,0.0,0,18.0,0.30,-0.24,-0.023333,1,NaN,No,Yes,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,0,Zero,0
965,161300,69.0,male,0.0,0,10.0,0.67,-0.06,0.393333,0,NaN,Yes,Yes,zero,0 to 1,ASPS-family,ASPS,0.0,0.0,0,Zero,0
966,161301,50.0,male,0.0,0,13.0,0.75,0.62,1.273333,0,NaN,Yes,Yes,zero,0 to 1,ASPS-family,ASPS,1.0,1.0,1,Zero,>=1
977,163402,65.0,female,0.0,0,9.0,-0.52,-0.76,-0.913333,0,NaN,Yes,Yes,zero,0 to 1,ASPS-family,ASPS,0.0,0.0,1,Zero,0
1198,202200,76.0,female,0.0,0,10.0,-1.13,-1.08,-0.906667,1,NaN,Yes,Yes,zero,2 to 3,ASPS-family,ASPS,1.0,1.0,2,Zero,0
1199,202202,50.0,male,0.0,0,18.0,0.57,0.48,1.416667,0,NaN,No,Yes,zero,0 to 1,ASPS-family,ASPS,0.0,0.0,1,Zero,0
1261,205600,72.0,male,0.0,0,10.0,-1.76,-2.10,-1.770000,1,NaN,Yes,Yes,zero,0 to 1,ASPS-family,ASPS,0.0,0.0,1,Zero,0
1276,206401,49.0,male,0.0,0,10.0,0.52,1.02,0.880000,0,NaN,Yes,No,zero,0 to 1,ASPS-family,ASPS,0.0,0.0,1,Zero,0


In [13]:
dementia_data[dementia_data["Global"].isnull()]

,ID,age,gender,dementia,dementia_all,educationyears,EF,PS,Global,diabetes,smoking,hypertension,hypercholesterolemia,lacunes_num,fazekas_cat,study,study1,SVD Simple Score,SVD Amended Score,Fazekas,lac_count,CMB_count
121,1,73.626283,male,0.0,0,10.0,NaN,NaN,NaN,0,current-smoker,Yes,Yes,more-than-zero,2 to 3,rundmc,rundmc,3.0,6.0,3,3 to 5,>=1
126,103,56.134155,male,0.0,0,10.0,NaN,NaN,NaN,0,current-smoker,No,No,zero,0 to 1,rundmc,rundmc,0.0,1.0,1,Zero,0
131,108,76.358658,male,0.0,0,15.0,NaN,NaN,NaN,0,ex-smoker,Yes,No,zero,0 to 1,rundmc,rundmc,0.0,1.0,1,Zero,0
132,109,68.536619,female,0.0,0,8.0,NaN,NaN,NaN,0,current-smoker,No,No,zero,0 to 1,rundmc,rundmc,0.0,1.0,1,Zero,0
135,111,71.501711,male,0.0,0,7.0,NaN,NaN,NaN,0,ex-smoker,Yes,Yes,zero,2 to 3,rundmc,rundmc,1.0,2.0,2,Zero,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,921,53.000000,male,0.0,0,10.0,0.57,NaN,NaN,0,current-smoker,No,Yes,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,0,Zero,0
1800,930,78.000000,female,0.0,0,9.0,-1.22,-1.08,NaN,0,never-smoker,No,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,1,Zero,0
1805,937,77.000000,male,0.0,0,10.0,0.19,NaN,NaN,0,never-smoker,Yes,Yes,zero,2 to 3,ASPS-elderly,ASPS,NaN,NaN,3,Zero,0
1814,950,77.000000,male,0.0,0,9.0,NaN,-0.89,NaN,0,never-smoker,No,No,zero,2 to 3,ASPS-elderly,ASPS,NaN,NaN,3,Zero,0


In [14]:
dementia_data.isna().sum()

ID                        0
age                       0
gender                    0
dementia                 34
dementia_all              0
educationyears            0
EF                      208
PS                      268
Global                  308
diabetes                  0
smoking                  11
hypertension              0
hypercholesterolemia      0
lacunes_num               0
fazekas_cat               0
study                     0
study1                    0
SVD Simple Score        677
SVD Amended Score       677
Fazekas                   0
lac_count                 0
CMB_count                 0
dtype: int64

## <font color=green> 3. Cleaning the dataset </font> ##

(This is the changes we made in the exercise April 19, 2024). 

In [15]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import asarray

In [16]:
dementia_data

,ID,age,gender,dementia,dementia_all,educationyears,EF,PS,Global,diabetes,smoking,hypertension,hypercholesterolemia,lacunes_num,fazekas_cat,study,study1,SVD Simple Score,SVD Amended Score,Fazekas,lac_count,CMB_count
0,1,52.67,male,0.0,0,11.0,-2.403333,-1.290000,-1.287000,0,current-smoker,Yes,Yes,more-than-zero,2 to 3,scans,scans,3.0,7.0,3,>5,>=1
1,10,64.58,male,0.0,0,10.0,1.280000,0.360000,0.744000,0,ex-smoker,Yes,Yes,more-than-zero,0 to 1,scans,scans,2.0,3.0,1,1 to 2,>=1
2,100,74.92,male,0.0,0,8.0,-1.440000,-1.520000,-0.922000,0,never-smoker,Yes,Yes,more-than-zero,0 to 1,scans,scans,1.0,2.0,1,1 to 2,0
3,101,74.83,male,1.0,1,9.0,NaN,-2.136271,-1.301102,0,current-smoker,Yes,Yes,more-than-zero,2 to 3,scans,scans,2.0,4.0,2,3 to 5,0
4,102,79.25,male,0.0,0,10.0,-0.920000,-1.493333,-0.924000,0,ex-smoker,Yes,Yes,more-than-zero,2 to 3,scans,scans,2.0,3.0,2,1 to 2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,989,67.00,female,0.0,0,13.0,-0.100000,-0.020000,-0.260000,0,never-smoker,Yes,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,0,Zero,0
1838,990,71.00,female,0.0,0,13.0,-0.370000,-1.060000,-1.003333,0,never-smoker,Yes,Yes,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,1,Zero,0
1839,991,55.00,male,0.0,0,10.0,0.460000,0.960000,0.610000,0,ex-smoker,No,Yes,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,0,Zero,0
1840,995,54.00,male,0.0,0,10.0,0.190000,0.030000,0.590000,0,ex-smoker,No,Yes,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,1,Zero,0


In [17]:
dementia_data_type = dementia_data['hypertension'].dtype
print(dementia_data_type)


object



In stead of having object in the hypertension column, we will change it to an integer by using the following code, and we do the same with the hypercholesterolemia-column 

In [18]:
dementia_data.replace({"hypertension" : {"No": 0, "Yes": 1}}, inplace=True)
dementia_data.replace({"hypercholesterolemia" : {"No": 0, "Yes": 1}}, inplace=True)
dementia_data.replace({"smoking" : {"never-smoker": int(0), "ex-smoker": int(1), "current-smoker": int(2)}}, inplace=True)
dementia_data.head()
dementia_data


/var/folders/_7/qfzsb2m168gd8fj4j2bnvgn00000gn/T/ipykernel_1380/701130176.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dementia_data.replace({"hypertension" : {"No": 0, "Yes": 1}}, inplace=True)
/var/folders/_7/qfzsb2m168gd8fj4j2bnvgn00000gn/T/ipykernel_1380/701130176.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dementia_data.replace({"hypercholesterolemia" : {"No": 0, "Yes": 1}}, inplace=True)
/var/folders/_7/qfzsb2m168gd8fj4j2bnvgn00000gn/T/ipykernel_1380/701130176.py:3: FutureWarning: Downcasting behavior in `repl

,ID,age,gender,dementia,dementia_all,educationyears,EF,PS,Global,diabetes,smoking,hypertension,hypercholesterolemia,lacunes_num,fazekas_cat,study,study1,SVD Simple Score,SVD Amended Score,Fazekas,lac_count,CMB_count
0,1,52.67,male,0.0,0,11.0,-2.403333,-1.290000,-1.287000,0,2.0,1,1,more-than-zero,2 to 3,scans,scans,3.0,7.0,3,>5,>=1
1,10,64.58,male,0.0,0,10.0,1.280000,0.360000,0.744000,0,1.0,1,1,more-than-zero,0 to 1,scans,scans,2.0,3.0,1,1 to 2,>=1
2,100,74.92,male,0.0,0,8.0,-1.440000,-1.520000,-0.922000,0,0.0,1,1,more-than-zero,0 to 1,scans,scans,1.0,2.0,1,1 to 2,0
3,101,74.83,male,1.0,1,9.0,NaN,-2.136271,-1.301102,0,2.0,1,1,more-than-zero,2 to 3,scans,scans,2.0,4.0,2,3 to 5,0
4,102,79.25,male,0.0,0,10.0,-0.920000,-1.493333,-0.924000,0,1.0,1,1,more-than-zero,2 to 3,scans,scans,2.0,3.0,2,1 to 2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1837,989,67.00,female,0.0,0,13.0,-0.100000,-0.020000,-0.260000,0,0.0,1,1,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,0,Zero,0
1838,990,71.00,female,0.0,0,13.0,-0.370000,-1.060000,-1.003333,0,0.0,1,1,zero,0 to 1,ASPS-elderly,ASPS,NaN,NaN,1,Zero,0
1839,991,55.00,male,0.0,0,10.0,0.460000,0.960000,0.610000,0,1.0,0,1,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,0,Zero,0
1840,995,54.00,male,0.0,0,10.0,0.190000,0.030000,0.590000,0,1.0,0,1,zero,0 to 1,ASPS-elderly,ASPS,0.0,0.0,1,Zero,0


In [19]:
dementia_data.smoking

0       2.0
1       1.0
2       0.0
3       2.0
4       1.0
       ... 
1837    0.0
1838    0.0
1839    1.0
1840    1.0
1841    1.0
Name: smoking, Length: 1842, dtype: float64

When exploring the dataset in step 2, we found that there were 11 missing values within the feature 'smoking', hence, it is not possible to transform the datatype of this feature into integers. This is why this feature consists of the datatype float64

<font color=red> (Jeg har forsøgt at lave den om til integers, men fik den fejlmelding, som beskrevet ovenfor, kan ikke gennemskue om det er essentielt eller ej)</font>

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer